### LangChain - Custom Agents (Custom Tools, Built-in Tools, Custom Agents and Out-of-the-Box Agents from LangGraph)

In [1]:
import os

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.utilities import GoogleSerperAPIWrapper

In [2]:
load_dotenv()

openai_api_key = os.environ["OPENAI_API_KEY"]
model_name = "gpt-4o"
temperature = 0.1
max_tokens = 1000

llm = ChatOpenAI(
    model = model_name,
    temperature=temperature,
    max_tokens=max_tokens,
    openai_api_key = openai_api_key
)

In [3]:
@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiplies two integers together"""
    
    return first_int * second_int

@tool
def add(first_int: int, second_int: int) -> int:
    """Sums two integers together"""
    
    return first_int + second_int

@tool
def exponentize(first_int: int, second_int: int) -> int:
    """Exponentize the base to the exponent value"""
    
    return first_int ** second_int

In [4]:
@tool("GoogleSearch")
def search(query_string: str):
    """
    Useful to search for any kinds of information and
    when you need to search the internet for any kinds of information, use this tool.
    """
    
    search = GoogleSerperAPIWrapper()
    
    return search.run(query_string)

In [5]:
tools = [add, multiply, exponentize, search]

In [6]:
from langchain_community.utilities import SQLDatabase

db_file_path = "./chinook.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_file_path}")

In [7]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db = db, llm = llm)

db_tools = toolkit.get_tools()

db_tools

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001E33906F690>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001E33906F690>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001E33906F690>),
 QuerySQLCheckerTool(description='Use this tool to 

In [8]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage, SystemMessage

SQL_PREFIX="""
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables.
"""

In [9]:
final_tools = tools + db_tools

In [10]:
final_tools

[StructuredTool(name='add', description='Sums two integers together', args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x000001E33451CFE0>),
 StructuredTool(name='multiply', description='Multiplies two integers together', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x000001E331283740>),
 StructuredTool(name='exponentize', description='Exponentize the base to the exponent value', args_schema=<class 'langchain_core.utils.pydantic.exponentize'>, func=<function exponentize at 0x000001E3379660C0>),
 StructuredTool(name='GoogleSearch', description='Useful to search for any kinds of information and\nwhen you need to search the internet for any kinds of information, use this tool.', args_schema=<class 'langchain_core.utils.pydantic.GoogleSearch'>, func=<function search at 0x000001E336C6DF80>),
 QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database.

In [11]:
system_message = SystemMessage(content=SQL_PREFIX)

agent_executor = create_react_agent(
    llm,
    tools = final_tools,
    state_modifier = system_message
)

In [12]:
question = """
take 3 to the fifth power and multiply that result by the sum of twelve and three, then square the whole result, 
and also let me know the capital of Malaysia and best places to visit at the capital ...
and also need 5 bulleted points about Mahatma Gandhi Achievements
and Which country's customers have spent the most in the sales?
"""

In [13]:
for stream in agent_executor.stream({
    "messages": [
        HumanMessage(content=question)
    ]
}):
    print(stream)
    print("***********")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_MV2hgrL9bfabxdUfibtMXWP9', 'function': {'arguments': '{"first_int": 3, "second_int": 5}', 'name': 'exponentize'}, 'type': 'function'}, {'id': 'call_guiT3D450fYGgm4Jlf5ch4F0', 'function': {'arguments': '{"first_int": 12, "second_int": 3}', 'name': 'add'}, 'type': 'function'}, {'id': 'call_TJ1L1R6DBZ3fHXE7aTqw7MaU', 'function': {'arguments': '{"query_string": "capital of Malaysia and best places to visit"}', 'name': 'GoogleSearch'}, 'type': 'function'}, {'id': 'call_TVj57JbhhVng67AWZFrc3E2U', 'function': {'arguments': '{"query_string": "Mahatma Gandhi Achievements"}', 'name': 'GoogleSearch'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 98, 'prompt_tokens': 738, 'total_tokens': 836, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'